## Scipy and Optimization

In the last notebook we took a tour of the `numpy` and `scipy` stack as it relates to math and statistics. This notebook is going to focus on another subset of `scipy` functionality that is absolutely critical to data analytics. Nearly every statistical algorithm aside from Ordinary Least Squares (OLS) regression cannot be solved algebraically. Because they cannot be solved algebraically, they must be solved **numerically**.

### Numeric Optimization

Let's start with an example of optimization. Let's say that you know the demand function for tickets to watch a local sports franchise. You can write the inverse demand function as

$$ P = 300 - \frac{1}{2}Q $$

and the total cost function as

$$ TC = 4000 + 45Q $$

In order to choose the right number of tickets to sell, you need to calculate the quantity of tickets that will maximize profits. We can calculate total revenue as $ TR = P \times Q $, and we can calculate profit as $ \Pi = TR - TC $. This means that our profit function is

$$ \Pi = 300Q - \frac{1}{2}Q^2 - 4000 - 45Q $$

In order to find the $Q$ associated with the highest achievable level of profit, we can use calculus to find the point at which the rate of change in the profit function is zero ($\frac{\partial\Pi}{\partial Q}=0$).

$$ \frac{\partial\Pi}{\partial Q} = 300 - Q - 45 = 0 \implies Q = 255$$

So we can **algebraically** solve this particular problem. This isn't always the case. Using `scipy`, we can solve this same problem, as well as many algebraically intractable problems that might be more interesting to us.


<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/paraboloid.png" width="200" height="200" />


In any optimization problem, we need to find a way to get ourselves to the minimum, and to know when we get there. When we look at the above image, we are able to visually trace the functional shape (looks like a rainbow ice cream cone to me...) and locate the bottom of the function. What we want to do is utilize an algorithm to "trace" our way from an arbitrary starting point within a function to the optimal point in that function.

In three or fewer dimensions, this is easy. Regressions and statistical models often live in worlds with 100's or 1000's (even millions sometimes) of dimensions. We can't visualize our way to the bottom of those functions!

The class of algorithm that is used to solve these problems is called **gradient descent**.

<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/gradDesc.png" width="400" />

**Gradient descent** is an algorithm that explores the shape of the function, and determines which direction is most likely to lead to the optimal point. Let's focus on minimization. We want to find our way to the *bottom* of a function, and we can use gradient descent to try to get there. Given any starting point, our goal is to check the direction in which we can move downward most quickly, and start moving in that direction. At some distance from our starting point, we will stop and re-evaluate the direction in which we would like to travel. Are we still heading downhill in the steepest direction? If we aren't, then we need to update our behavior.

Our gradient descent algorithm will keep "looking around" and moving down until it reaches a point at which it can no longer move "down" in any meaningful way. That is the stopping point, and is treated as the optimum.


With an intuitive understanding of how optimization will happen computationally, it's time to learn a bit more about the math and the code that will help us to achieve computational optimization.

Consider a function, $f$, with two variables $x$ and $y$. Because there are two input variables in the function, it has two partial derivatives:

$$ \frac{\partial f}{\partial x} \text{ and } \frac{\partial f}{\partial y} $$

Each partial derivative tells us how $f$ changes as we move in a particular dimension **all else equal**. The **gradient**, then, is the vector of all partial derivatives of a given function at any point along the function:

$$ \nabla f = \left[ \begin{matrix} \frac{\partial f}{\partial x} \\ \\ \frac{\partial f}{\partial y} \end{matrix} \right]  $$

We can use the gradient to determine the linear approximation of a function at any given point. Think about the gradient as the mathematical representation of the slope and direction of a hill you are hiking on. If we know the gradient, we know which way is down. As we continue to calculate gradients while walking, we can continue to ensure that we will walk downhill until we reach the bottom of the hill.




The steps of our gradient descent function will be the following:

- Evaluate the gradient of the function
- Find the direction of steepest descent
- Determine how far to move in that direction
- Move to new point
- Reevaluate the gradient
- Stop moving when gradient is within a margin of error from 0

Let's try to implement gradient descent by solving our old profit maximization problem computationally. The very first thing that we need to do is write a Python function that will represent our mathematical function.

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    return tr - tc

This function will allow us to calculate profit at any output level based on our assumed total costs and demand curve. With this function, we can quickly calculate the gradient (in this case, just a simple derivative because our function is univariate) by calculating profit at two nearby points, and dividing by the horizontal distance between those points:

In [ ]:
# Gradient at q=200

(profit(201) - profit(199))/2

    55.0



Thus, a one unit increase in output at $Q=200$ results in a $55 increase in profits. This is cool, but it isn't enough for us to find the point of maximum profit (the optimal point). For that, we will need to calculate LOTS of gradients in order to move along the function until we cannot increase profits any further.

Fortunately for us, `scipy` comes with optimization tools that will do all of the heavy lifting of the "search" for the optimal point. All that we need to do is frame our question algorithmically, and let `scipy` do the rest:

In [ ]:
import numpy as np
from scipy.optimize import minimize

We start by importing the `minimize` function from `scipy.optimize`. Hang on! Weren't we working on a MAXIMIZATION problem?? What are we doing here?

Maximization and minimization are the **same thing**. To maximize a function, you can multiply that function by `-1` and then calculate the minimum of the new "upside-down" function. It is functionally equivalent. So, in computational optimization, we always minimize.

### Prepping for optimization

As we prepare to optimize, there are two common problems with our function that we may need to resolve:

1) When using `minimize` we can only pass an array of inputs, so we have to be careful to write our function accordingly
2) Our problem is concave, and so has a maximum
	- We need to restate it as a minimization problem

Problem 1 does not apply, since our function in univariate. In order to make our problem a minimization problem, we can flip our profit maximization function. We will simply return -1 * profit:

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    pi =  tr - tc # didn't name it profit because that is our function's name! Don't want to clutter our name space!
    return -1*pi

### Making the call to `minimize`

Now that our function is ready, it is time to minimize! The `minimize` function takes two arguments:
1. Our function that we want to optimize
2. A starting guess (as a vector)

Let's give it a try.

In [ ]:
res = minimize(profit, [0]) # provide function and starting inputs
res

          fun: -28512.499999980355
     hess_inv: array([[1.00000175]])
          jac: array([0.])
      message: 'Optimization terminated successfully.'
         nfev: 21
          nit: 3
         njev: 7
       status: 0
      success: True
            x: array([255.00019821])



That's it! No calculus, no searching, no checking gradients manually. `minimize` simply takes our function and our starting guess and brings us back the optimal choice. We get lots of information stored in the attributes of the `res` object:

- `fun` provides the value of the function (this is -1 times the profit level at the optimal output in our example)
- `hess_inv` and `jac` are measures of gradient and are used to determine how far to go and in which direction
- `message` should be self-explanatory
- `nfev` is the number of times the function (in this case `profit`) was evaluated during the search
- `nit` is the number of iterations it took to find the optimum
- `njev` is the number of times the Jacobian was estimated
- `status` is a code associated with the `message` and `success` atrributes
- `success` tells you whether or not an optimum was found (sometimes it cannot be easily found!)
- `x` probably the most important attribute. This tells us the optimal input value (in our case $Q$), or set of values depending on our function. In a regression context, this could represent the fitted coefficients!

Going forward, you will realize (especially because so many of them print the output as they optimize) just how many libraries in Python use this minimize function behind the scenes. It is used in `statsmodels`, `sklearn`, and many other high-profile libraries! Now that you know where it really happens (in `scipy`!), you'll be better able to troubleshoot the problems that will inevitably arise as you use statistical models.

**Solve-it!**

In this lesson we learned about optimization using SciPy. For the assignment this week, I would like you to build off of your `RegressionModel` class. You will add a Logistic Regression (Logit) method to your class, so that when the `regression_type` parameter is `logit`, Logistic Regression Results are returned.

Your job is to create the following functionality within your class object:
- a method (call it `logistic_regression`) that estimates the results of logistic regression using your `x` and `y` data frames, and using a likelihood function and gradient descent (DO NOT USE PREBUILT REGRESSION FUNCTIONS).
    - You need to write a function to calculate the Log-likelihood of your model
    - You need to implement gradient descent to find the optimal values of beta
    - You need to use your beta estimates, the model variance, and calculate the standard errors of the coefficients, as well as Z statistics and p-values
    - the results should be stored in a dictionary named `results`, where each variable name (including the intercept if `create_intercept` is `True`) is the key, and the value is another dictionary, with keys for `coefficient`, `standard_error`, `z_stat`, and `p_value`. The coefficient should be the log odds-ratio (which takes the place of the coefficients in OLS)
- a method called `fit_model` that uses the `self.regression_type` attribute to determine whether or not to run an OLS or Logistic Regression using the data provided. This method should call the correct regression method.
- an updated method (call it `summary`) that presents your regression results in a table
    - Columns should be: Variable name, Log odds-ratio value, standard error, z-statistic, and p-value, in that order.
    - Your summary table should have different column titles for OLS and Logistic Regressions! (think if statement...)

You only need to define the class. My code will create an instance of your class (be sure all the names match these instructions, and those from last week!), and provide data to run a regression. I will provide the same data to you, so that you can experiment and make sure that your code is functioning properly.

NOTE: I have created a [primer on Logistic regression](https://github.com/dustywhite7/Econ8320/blob/master/SlidesPDF/9-2%20-%20Logit%20Primer.pdf) to go with this assignment. See the Github slidesPDF folder

Put the code that you would like graded in the cell labeled `#si-exercise`. I recommend copying your code from the last assignment (in chapter 9 about linear regression and `numpy`), and continuing from there.



In [1]:
#si-exercise
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.optimize import minimize

class RegressionModel(object):
  def __init__(self, x, y, create_intercept = True, regression_type = ['ols', 'logit']):
    if create_intercept:
      x['intercept'] = 1
    self.x = x
    self.y = y
    self.create_intercept = create_intercept
    self.regression_type = regression_type
    self.results = {}

  def add_intercept(self):
    if self.create_intercept == True:
        if 'intercept' not in self.x.columns:
            self.x['intercept'] = 1
            self.x = self.x.reset_index(drop=True)
        else:
            print("Warning: 'intercept' variable already present.")
    else:
        pass

  def fit_model(self):
    if 'ols' in self.regression_type:
        self.ols_regression()
    if 'logit' in self.regression_type:
        self.logistic_regression()

  def ols_regression(self):
    if self.create_intercept:
      self.add_intercept()

    beta = np.linalg.inv(self.x.T @ self.x) @ (self.x.T @ self.y)
    df = len(self.y) - self.x.shape[1]
    shat2 = ((self.y.T @ self.y) - ((self.y.T @ self.x) @ beta)) / df
    cov_beta = shat2 * np.linalg.inv(self.x.T @ self.x)
    variance = np.diag(cov_beta)
    stan_err = np.sqrt(variance)
    t_test = beta / stan_err
    p_val = t.sf(t_test, df)

    variable_names = list(self.x.columns)
    if self.create_intercept:
        if 'intercept' not in variable_names:
            variable_names.append('intercept')

    self.results = {}
    for i, var_name in enumerate(variable_names):
      self.results[var_name] = {
          "coefficient": beta[i].tolist(),
          "standard_error": stan_err[i].tolist(),
          "t_stat": t_test[i].tolist(),
          "p_value": p_val[i].tolist()
            }

  def logistic_regression(self):
    if self.create_intercept:
        self.add_intercept()

    starting_beta = np.zeros((self.x.shape[0]))

    def log_likelihood(beta):
      beta = beta.reshape((-1, 1))  # Ensure beta is a column vector
      linear_combination = np.sum(self.x * beta, axis=1)
      p = 1 / (1 + np.exp(-linear_combination))
      p = np.clip(p, 1e-10, 1 - 1e-10)  # Clip probabilities to prevent log(0) or log(1)
      likelihood = np.sum(self.y * np.log(p) + (1 - self.y) * np.log(1 - p))
      return -likelihood



    result = minimize(log_likelihood, starting_beta)

    beta = result.x

    p = (np.exp(self.x.T @ beta))/(1 + np.exp(self.x.T @ beta))

    sigma2 = (len(self.y)*(p * (1 - p)))

    serror = sigma2 * np.linalg.pinv(self.x.T @ self.x)

    zstat = beta / np.sqrt(sigma2)

    pval = stat.norm.sf(zstat)

    self.results = dict()

    for i in enumerate(self.x.columns):
        self.results[i[1]] = {
                'coefficient' : beta[i[0]],
                'standard_error' : serror[i[0]],
                't_stat' : zstat[i[0]],
                'p_value' : pval[i[0]]
                }


  def summary(self):
    if 'ols' in self.regression_type:
      variable_stats = {}
      for variable_name in self.x.columns:
          stats = self.results[variable_name]
          variable_stats[variable_name] = {
              'coefficient value': stats["coefficient"],
              'standard error': stats["standard_error"],
              't-statistic': stats["t_stat"],
              'p-value': stats["p_value"]
          }
      summary_df = pd.DataFrame(variable_stats).T
      summary_df.index.name = 'Variable name'
      summary_df.reset_index(inplace=True)
      summary_df.columns = ['Variable name', 'coefficient value', 'standard error', 't-statistic', 'p-value']
      print(summary_df)
      return summary_df

    elif 'logit' in self.regression_type:
      variable_stats = {}
      for variable_name in self.x.columns:
          stats = self.results[variable_name]
          variable_stats[variable_name] = {
              'Log odds-ratio value': stats["coefficient"],
              'standard error': stats["standard_error"],
              'z-statistic': stats["z_stat"],
              'p-value': stats["p_value"]
          }

      summary_df = pd.DataFrame(variable_stats).T
      summary_df.index.name = 'Variable name'
      summary_df.reset_index(inplace=True)
      summary_df.columns = ['Variable name', 'coefficient value', 'standard error', 'z-statistic', 'p-value']
      print(summary_df)
      return summary_df

In [3]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ECON 8320/Week 11 - Optimization/assignment8Data.csv'
data = pd.read_csv(file_path)
x = data[['sex','age','educ']]
y = data['white']
reg = RegressionModel(x, y, create_intercept=True, regression_type = ['logit'])
reg.fit_model()
reg.summary()

<ipython-input-1-b2b00f54cc42>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['intercept'] = 1


KeyboardInterrupt: 

In [ ]:
sex = {'coefficient': -1.1229156890097627,
      'standard_error': 0.39798772782618025,
      'z_stat': -2.821483202869492,
      'p_value': 0.004780214077269219}
age = {'coefficient': -0.007012518056833769,
        'standard_error': 0.010835821823286998,
        'z_stat': -0.6471607019011091,
        'p_value': 0.5175279421902776}
educ = {'coefficient': -0.046485475816343394,
        'standard_error': 0.10100278092776117,
        'z_stat': -0.46023956359766527,
        'p_value': 0.6453442758780246}
intercept = {'coefficient': 5.735435005488546,
            'standard_error': 1.1266207023561843,
            'z_stat': 5.090830475148922,
            'p_value': 3.56498650369634e-07}

In [2]:
import unittest
import json
import pandas as pd
import numpy as np
import sys as system
import io
import re
import pandas as pd
import os
import csv
from scipy.stats import t
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/ECON 8320/Week 11 - Optimization/assignment8Data.csv'
data = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Tests
class TestCase(unittest.TestCase):
  def testLogitCorrectValues(self):
        data = pd.read_csv(file_path)
        x = data.loc[:100, ['sex','age','educ']]
        y = data.loc[:100, 'white']
        reg = RegressionModel(x, y, create_intercept=True, regression_type='logit')
        reg.fit_model()

        sex = {'coefficient': -1.1229156890097627,
        'standard_error': 0.39798772782618025,
        'z_stat': -2.821483202869492,
        'p_value': 0.004780214077269219}
        age = {'coefficient': -0.007012518056833769,
        'standard_error': 0.010835821823286998,
        'z_stat': -0.6471607019011091,
        'p_value': 0.5175279421902776}
        educ = {'coefficient': -0.046485475816343394,
        'standard_error': 0.10100278092776117,
        'z_stat': -0.46023956359766527,
        'p_value': 0.6453442758780246}
        intercept = {'coefficient': 5.735435005488546,
        'standard_error': 1.1266207023561843,
        'z_stat': 5.090830475148922,
        'p_value': 3.56498650369634e-07}
        sexEq = (round(sex['coefficient'], 1)==round(reg.results['sex']['coefficient'], 1)) & (round(sex['standard_error'], 1)==round(reg.results['sex']['standard_error'], 1)) & (round(sex['z_stat'], 1)==round(reg.results['sex']['z_stat'], 1)) & (round(sex['p_value'], 1)==round(reg.results['sex']['p_value'], 1))
        ageEq = (round(age['coefficient'], 1)==round(reg.results['age']['coefficient'], 1)) &  (round(age['standard_error'], 1)==round(reg.results['age']['standard_error'], 1)) & (round(age['z_stat'], 1)==round(reg.results['age']['z_stat'], 1)) & (round(age['p_value'], 1)==round(reg.results['age']['p_value'], 1))
        educEq = (round(educ['coefficient'], 1)==round(reg.results['educ']['coefficient'], 1)) & (round(educ['standard_error'], 1)==round(reg.results['educ']['standard_error'], 1)) & (round(educ['z_stat'], 1)==round(reg.results['educ']['z_stat'], 1)) & (round(educ['p_value'], 1)==round(reg.results['educ']['p_value'], 1))
        interceptEq = (round(intercept['coefficient'], 1)==round(reg.results['intercept']['coefficient'], 1)) & (round(intercept['standard_error'], 1)==round(reg.results['intercept']['standard_error'], 1)) & (round(intercept['z_stat'], 1)==round(reg.results['intercept']['z_stat'], 1)) & (round(intercept['p_value'], 1)==round(reg.results['intercept']['p_value'], 1))

        sexEq2 = (round(sex['coefficient'], 1)==round(reg.results['sex']['coefficient'], 1)) & (round(sex['standard_error'], 1)==round(reg.results['sex']['standard_error'], 1)) & (round(sex['z_stat'], 1)==round(reg.results['sex']['z_stat'], 1)) & (round(sex['p_value']/2, 1)==round(reg.results['sex']['p_value'], 1))
        ageEq2 = (round(age['coefficient'], 1)==round(reg.results['age']['coefficient'], 1)) &  (round(age['standard_error'], 1)==round(reg.results['age']['standard_error'], 1)) & (round(age['z_stat'], 1)==round(reg.results['age']['z_stat'], 1)) & (round(age['p_value']/2, 1)==round(reg.results['age']['p_value'], 1))
        educEq2 = (round(educ['coefficient'], 1)==round(reg.results['educ']['coefficient'], 1)) & (round(educ['standard_error'], 1)==round(reg.results['educ']['standard_error'], 1)) & (round(educ['z_stat'], 1)==round(reg.results['educ']['z_stat'], 1)) & (round(educ['p_value']/2, 1)==round(reg.results['educ']['p_value'], 1))
        interceptEq2 = (round(intercept['coefficient'], 1)==round(reg.results['intercept']['coefficient'], 1)) & (round(intercept['standard_error'], 1)==round(reg.results['intercept']['standard_error'], 1)) & (round(intercept['z_stat'], 1)==round(reg.results['intercept']['z_stat'], 1)) & (round(intercept['p_value']/2, 1)==round(reg.results['intercept']['p_value'], 1))

        self.assertTrue((sexEq & ageEq & educEq & interceptEq)|(sexEq2 & ageEq2 & educEq2 & interceptEq2), "Your coefficients are not corretly calculated.")

In [ ]:
class TestCase(unittest.TestCase):

    def testSummaryTableOutput(self):
      stdout = system.stdout
      system.stdout = io.StringIO()


      data = pd.read_csv(file_path)
      x = data.loc[:100, ['sex','age','educ']]
      y = data.loc[:100, 'white']
      reg = RegressionModel(x, y, create_intercept=True, regression_type='logit')
      reg.fit_model()
      reg.summary()

      output = system.stdout.getvalue()
      system.stdout = stdout

      labels = bool(re.findall(r'[Cc]oef.*[Ss]t.*[Zz].*[Pp].*', output))
      sex = bool(re.findall(r'sex.*\d+.*\d+.*\d+.*\d+.*', output))
      age = bool(re.findall(r'age.*\d+.*\d+.*\d+.*\d+.*', output))
      educ = bool(re.findall(r'educ.*\d+.*\d+.*\d+.*\d+.*', output))
      intercept = bool(re.findall(r'intercept.*\d+.*\d+.*\d+.*\d+.*', output))

      self.assertTrue(labels&sex&age&educ&intercept, "Your table is not correctly formatted.")

Your job is to create the following functionality within your class object:
- a method (call it `logistic_regression`) that estimates the results of logistic regression using your `x` and `y` data frames, and using a likelihood function and gradient descent (DO NOT USE PREBUILT REGRESSION FUNCTIONS).
    - You need to write a function to calculate the Log-likelihood of your model
    - You need to implement gradient descent to find the optimal values of beta
    - You need to use your beta estimates, the model variance, and calculate the standard errors of the coefficients, as well as Z statistics and p-values
    - the results should be stored in a dictionary named `results`, where each variable name (including the intercept if `create_intercept` is `True`) is the key, and the value is another dictionary, with keys for `coefficient`, `standard_error`, `z_stat`, and `p_value`. The coefficient should be the log odds-ratio (which takes the place of the coefficients in OLS)
- a method called `fit_model` that uses the `self.regression_type` attribute to determine whether or not to run an OLS or Logistic Regression using the data provided. This method should call the correct regression method.
- an updated method (call it `summary`) that presents your regression results in a table
    - Columns should be: Variable name, Log odds-ratio value, standard error, z-statistic, and p-value, in that order.
    - Your summary table should have different column titles for OLS and Logistic Regressions! (think if statement...)

In [43]:
import numpy as np
import scipy.stats as stat
import pandas as pd
from scipy.optimize import minimize

class RegressionModel(object):
  def __init__(self, x, y, create_intercept=True, regression_type= ['ols', 'logit']):
      self.results = {}
      if isinstance(x, pd.DataFrame):
          self.x = x
      else:
          raise RuntimeError("Matrix 'x' is not a DataFrame.")
      if isinstance(y, pd.DataFrame) | isinstance(y, pd.Series):
          self.y = y
      else:
          raise RuntimeError("Matrix 'y' is not a DataFrame.")
      if isinstance(create_intercept, bool):
          self.create_intercept = create_intercept
          if self.create_intercept:
              self.add_intercept()
      else:
          raise RuntimeError("Parameter 'create_intercept' must be a boolean value.")
      if isinstance(regression_type, str):
          if regression_type=="ols":
              self.regression_type=regression_type
          elif regression_type=='logit':
              self.regression_type=regression_type
          else:
              raise RuntimeError("Only OLS and Logistic regressions ('ols' or 'logit', respectively) are supported")
      else:
          raise RuntimeError("Parameter 'regression_type' must be a string with value 'ols' or 'logit'.")

  def fit_model(self):
      if 'ols' in self.regression_type:
          self.ols_regression()
      elif 'logit' in self.regression_type:
          self.logistic_regression()
          print("Success in logit choice")
      else:
          print("No valid regression type chosen")

  def add_intercept(self):
      self.x = self.x.assign(intercept=pd.Series([1]*np.shape(self.x)[0]))

  def ols_regression(self):
    x = self.x
    y = self.y
    n, k = np.shape(x)
    beta = np.dot(np.linalg.solve(x.T.dot(x), np.eye(k)), x.T.dot(y))
    eps = y - x.dot(beta)
    shat = eps.T.dot(eps)/(n-k)
    covar = shat * np.linalg.solve(x.T.dot(x), np.eye(k))
    var = np.diag(covar)
    serror = np.asarray([np.sqrt(i) for i in var])
    tstat = np.asarray([i[1]/serror[i[0]] for i in enumerate(beta)])
    pval = stat.t.sf(tstat, n-k)

    self.results = dict()

    for i in enumerate(self.x.columns):
        self.results[i[1]] = {
                'coefficient' : beta[i[0]],
                'standard_error' : serror[i[0]],
                't_stat' : tstat[i[0]],
                'p_value' : pval[i[0]]
                }
  def logistic_regression(self):
    if self.create_intercept:
        self.add_intercept()

    starting_beta = np.zeros((self.x.shape[0]))

    def log_likelihood(beta):
      beta = beta.reshape((-1, 1))  # Ensure beta is a column vector
      linear_combination = np.sum(self.x * beta, axis=1)
      p = 1 / (1 + np.exp(-linear_combination))
      p = np.clip(p, 1e-10, 1 - 1e-10)  # Clip probabilities to prevent log(0) or log(1)
      likelihood = np.sum(self.y * np.log(p) + (1 - self.y) * np.log(1 - p))
      return -likelihood



    result = minimize(log_likelihood, starting_beta)

    beta = result.x

    p = (np.exp(self.x.T @ beta))/(1 + np.exp(self.x.T @ beta))

    sigma2 = (len(self.y)*(p * (1 - p)))

    serror = sigma2 * np.linalg.pinv(self.x.T @ self.x)

    zstat = beta / np.sqrt(sigma2)

    pval = stat.norm.sf(zstat)

    self.results = dict()

    for i in enumerate(self.x.columns):
        self.results[i[1]] = {
                'coefficient' : beta[i[0]],
                'standard_error' : serror[i[0]],
                't_stat' : zstat[i[0]],
                'p_value' : pval[i[0]]
                }

  def summary(self):
    if 'ols' in self.regression_type:
      print("Variable Name".ljust(25) + "| Coefficient".ljust(15) + "| Standard Error".ljust(17) + "| T-Statistic".ljust(15) + "| P-Value".ljust(15) + "\n" + "-"*85)
      for i in self.results:
          print("{}".format(i).ljust(25) + "| {}".format(round(self.results[i]['coefficient'], 3)).ljust(15) + "| {}".format(round(self.results[i]['standard_error'], 3)).ljust(17) + "| {}".format(round(self.results[i]['t_stat'], 3)).ljust(15) + "| {}".format(round(self.results[i]['p_value'], 3)).ljust(15))

    if 'logit' in self.regression_type:
      print("Variable Name".ljust(25) + "| Coefficient".ljust(15) + "| Standard Error".ljust(17) + "| Z-Statistic".ljust(15) + "| P-Value".ljust(15) + "\n" + "-"*85)
      for i in self.results:
          print("{}".format(i).ljust(25) + "| {}".format(round(self.results[i]['coefficient'], 3)).ljust(15) + "| {}".format(round(self.results[i]['standard_error'], 3)).ljust(17) + "| {}".format(round(self.results[i]['z_stat'], 3)).ljust(15) + "| {}".format(round(self.results[i]['p_value'], 3)).ljust(15))

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ECON 8320/Week 11 - Optimization/assignment8Data.csv'
data = pd.read_csv(file_path)
x = data[['sex','age','educ']]
y = data['white']
reg = RegressionModel(x, y, create_intercept=True, regression_type = 'logit')
reg.fit_model()
reg.summary()

In [ ]:
sex = {'coefficient': -1.1229156890097627,
      'standard_error': 0.39798772782618025,
      'z_stat': -2.821483202869492,
      'p_value': 0.004780214077269219}
age = {'coefficient': -0.007012518056833769,
        'standard_error': 0.010835821823286998,
        'z_stat': -0.6471607019011091,
        'p_value': 0.5175279421902776}
educ = {'coefficient': -0.046485475816343394,
        'standard_error': 0.10100278092776117,
        'z_stat': -0.46023956359766527,
        'p_value': 0.6453442758780246}
intercept = {'coefficient': 5.735435005488546,
            'standard_error': 1.1266207023561843,
            'z_stat': 5.090830475148922,
            'p_value': 3.56498650369634e-07}